In [1]:
import os
from entmoot.space.space import Space
from entmoot.optimizer.gurobi_utils import get_core_gurobi_model
os.chdir("..")
from src.models.blackbox_sm_fnn import *

In [3]:
# Load surrogate model
func = SurrogateModelFNN()
# initialize the search space manually
space = Space(func.get_bounds())

In [4]:
# get bounds of model
func.get_bounds()

[Integer(low=0, high=1, prior='uniform', transform='identity'),
 Integer(low=0, high=1, prior='uniform', transform='identity'),
 (0.0, 1.0),
 (0.0, 1.0)]

In [5]:
# get the core of the gurobi model from helper function 'get_core_gurobi_model'
core_model = get_core_gurobi_model(space)

# ordering of variable indices is dependent on space definition 


Academic license - for non-commercial use only - expires 2021-06-18
Using license file /Users/jani/gurobi.lic


In [6]:
x0 = core_model._cont_var_dict[0]
x1 = core_model._cont_var_dict[1]
x2 = core_model._cont_var_dict[2]
x3 = core_model._cont_var_dict[3]

In [7]:
core_model.addConstr(x0 + x1 == 1, name="hi")
core_model.addConstr(x2 == 0.5, name="hallo")

<gurobi.Constr *Awaiting Model Update*>

In [8]:
core_model.update()
core_model

<gurobi.Model MIP instance Unnamed: 2 constrs, 4 vars, No parameter changes>

In [9]:
func.get_bounds()

[Integer(low=0, high=1, prior='uniform', transform='identity'),
 Integer(low=0, high=1, prior='uniform', transform='identity'),
 (0.0, 1.0),
 (0.0, 1.0)]

In [10]:
from entmoot.optimizer.entmoot_minimize import entmoot_minimize

# cont_var_dict contains all continuous variables


# define constraints accordingly
core_model.addConstr(x0 + x1 == 1)
core_model.addConstr(x2 == 0.5)

# specify the model core in `acq_optimizer_kwargs`
res = entmoot_minimize(
    func,
    func.get_bounds(),
    n_calls=15,
    n_points=10000,
    base_estimator="GBRT",
    std_estimator="L1DDP", 
    n_initial_points=10, 
    initial_point_generator="random",
    acq_func="LCB", 
    acq_optimizer="global",
    x0=None,
    y0=None,  
    random_state=100, 
    acq_func_kwargs=None, 
    acq_optimizer_kwargs={
      "add_model_core": core_model
    },
    std_estimator_kwargs=None,
    model_queue_size=None,
    base_estimator_kwargs={
        "min_child_samples": 2
    },
    verbose = True,
)


SOLVER: start solution process...

SOLVER: generate  10 initial points...

 itr_1
   new point obj.: 1.93843
   best obj.:       1.93843

 itr_2
   new point obj.: 0.34318
   best obj.:       1.93843

 itr_3
   new point obj.: 1.32288
   best obj.:       0.34318

 itr_4
   new point obj.: 2.14766
   best obj.:       0.34318

 itr_5
   new point obj.: 2.31324
   best obj.:       0.34318

 itr_6
   new point obj.: 0.20564
   best obj.:       0.34318

 itr_7
   new point obj.: 2.33174
   best obj.:       0.20564

 itr_8
   new point obj.: 3.2189
   best obj.:       0.20564

 itr_9
   new point obj.: 2.7112
   best obj.:       0.20564

 itr_10
   new point obj.: 1.72345
   best obj.:       0.20564

SOLVER: initial points exhausted
   -> switch to model-based optimization

 itr_11
   new point obj.: 2.02948
   best obj.:       0.20564

 itr_12
   new point obj.: 2.02948
   best obj.:       0.20564


/Users/jani/.conda/envs/animate_logos/lib/python3.8/site-packages/entmoot/optimizer/optimizer.py:518: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/jani/.conda/envs/animate_logos/lib/python3.8/site-packages/entmoot/optimizer/optimizer.py:518: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "



 itr_13
   new point obj.: 2.02948
   best obj.:       0.20564

 itr_14
   new point obj.: 2.02793
   best obj.:       0.20564

 itr_15
   new point obj.: 2.02716
   best obj.:       0.20564

SOLVER: finished run!
SOLVER: best obj.: 0.20564



In [11]:
res

           fun: 0.20564211151127337
     func_vals: array([1.93842887, 0.34317826, 1.32288254, 2.14765963, 2.3132404 ,
       0.20564211, 2.33174251, 3.21890013, 2.71119803, 1.72345434,
       2.02947809, 2.02947809, 2.02947809, 2.02793028, 2.02715637])
 gurobi_mipgap: [0.0, 0.0, 0.0, 0.0, 0.0]
      model_mu: [array([1.95616533]), array([2.04448004]), array([2.03384405]), array([1.95117299]), array([2.02945069])]
     model_std: [array([-0.33042684]), array([-0.]), array([-0.]), array([-0.00763591]), array([-0.00395297])]
        models: [EntingRegressor(base_estimator=LGBMRegressor(min_child_samples=2,
                                             objective='regression',
                                             random_state=186422792,
                                             verbose=-1),
                random_state=186422792,
                std_estimator=<entmoot.learning.distance_based_std.DistanceBasedPenalty object at 0x7ff027e2c460>), EntingRegressor(base_estimator=LGBMR